In [1]:
import guidance
from llama_quantized import LLaMAQuantized
from search import web_search, find_closest_sections

In [2]:
model = LLaMAQuantized('Wizard-Vicuna-13B-Uncensored-GPTQ')

Found the following quantized model: /home/models/Wizard-Vicuna-13B-Uncensored-GPTQ/Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order.safetensors


/opt/conda/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:


Model to device
Model to device
Loading tokenizer from: /home/models/Wizard-Vicuna-13B-Uncensored-GPTQ/


In [3]:
search_result = web_search('tallest building in Seattle')
search_result
scored_results = find_closest_sections(search_result, 'tallest building in Seattle')
scored_results

Reading https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Seattle...
Reading https://www.re-thinkingthefuture.com/2020/06/10/a1006-15-tallest-buildings-in-seattle/...
Reading https://en.wikipedia.org/wiki/Columbia_Center...
Reading https://www.skyscrapercenter.com/city/seattle...
Skipping (not enough text)
Reading https://en.wikipedia.org/wiki/Space_Needle...
Reading https://seattle.curbed.com/maps/mapping-seattles-tallest-building-over-history...
found 141 sections
[i] tensor([0.8764], device='cuda:0'): The 76-story structure is the tallest building in Seattle and the state of Washington, reaching a height of 933 ft (284 m).
[i] tensor([0.8525], device='cuda:0'): Photo: Visitor7/Wikimedia Commons. If and when it's completed, the proposed 77-story tower at Third & Marion will become the tallest building in Seattle, replacing the 76-story Columbia Center building. That got us thinking about the ever-changing Seattle skyline and how it has evolved over time.
[i] tensor([0.8345],

[{'text': 'The 76-story structure is the tallest building in Seattle and the state of Washington, reaching a height of 933 ft (284 m).',
  'score': 0.8763927817344666},
 {'text': "Photo: Visitor7/Wikimedia Commons. If and when it's completed, the proposed 77-story tower at Third & Marion will become the tallest building in Seattle, replacing the 76-story Columbia Center building. That got us thinking about the ever-changing Seattle skyline and how it has evolved over time.",
  'score': 0.8525447845458984},
 {'text': 'Standing at 284m height, Columbia Center in downtown Seattle, Washington holds the title as the tallest building in Seattle and the state of Washington since 1985.',
  'score': 0.8345247507095337},
 {'text': '[2][3] An additional 65 high-rise buildings are under construction or undergoing planning and design review, as of 2016 . [4]. The tallest building in Seattle is the 76-story Columbia Center, which rises 937 feet (286 m) and was completed in 1985.',
  'score': 0.82293

In [4]:
demo_results = [{'text': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'text': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'},
 {'text': 'Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]'}]
practice_round = guidance(
'''{{#user~}}
Who are the founders of OpenAI?
{{~/user}}
{{#assistant~}}
<search>OpenAI founders</search>
{{~/assistant}}
{{#user~}}
Search results:
{{~#each results}}
<result>
{{this.text}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.
{{~/assistant}}''', model)
practice_round = practice_round(results=demo_results)

def is_search(completion):
    return '<search>' in completion

def search(query):
    search_results = web_search(query)
    scored_results = find_closest_sections(search_results, query)
    scored_results = scored_results[:4]
    print(scored_results)
    return scored_results

prompt = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round.
{{~/assistant}}
{{>practice_round}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
{{#user~}}
{{user_query}}
{{~/user}}
{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}
{{#if (is_search query)}}
{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.text}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
{{/if}}''', model)

prompt = prompt(practice_round=practice_round, search=search, is_search=is_search, user_query="What are some great places to see cherry blossoms in Seattle?")

Stop program system You are a helpful assistant. user From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond. assistant Ok, I will do that. Let's do a practice round <span style='background-color: rgba(165, 165, 165, 0.1); display: inline;' title='{{#block 'practice_round'}}USER: Who are the founders of OpenAI?
ASSISTANT: <search>OpenAI founders</search></s>
USER: Search results:
<result>
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result>
ASSISTANT: The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.</s>{{/block}}'>USER: Who are the founders of OpenAI?
ASSISTANT: <search>OpenAI founders</search></s>
USER: Search results:
<result>
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]
</result>
ASSISTANT: The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.</s> user That was great, now let's do another one. assistant Ok, I'm ready. user What are some great places to see cherry blossoms in Seattle? assistant <search>cherry blossoms Seattle </search> user Search results: 
<result>
 Hot tip: you can also use this map of Seattle trees that was created by the Seattle Department of Transportation (SDOT) to find cherry blossoms near you. As you can see on the legend in the upper right corner, the blue dots represent the Prunus (cherry/plum) trees in the city. 
</result>
<result>
 The University of Washington is perhaps the most famous place in Seattle for cherry blossom viewing every spring. Starting around mid-March, you’ll want to head over to the university to see the quad explode with pink blossoms. For updates you can follow the UW Cherry Blossoms Twitter account. 2. 
</result>
<result>
 You can also find cherry blossoms in your own neighborhood; scroll to the end of this article fo